# Heterodyne Measurements

The goal of this is to try to perform heterodyne measurements for states in the Fock basis with Xanadu's Strawberry Fields. Xanadu's Fock basis does not provide a way to perform heterodyne measurement in the Fock basis, but since it seems like this is applying the following operator: $\frac{1}{\pi}|\alpha><\alpha|\psi>$. If I am not mistaken, $<\alpha|\psi>$ should produce some scalar $x$, so the entire thing should produce $\frac{x}{\pi}|\alpha>$. The only observable component is $\frac{x}{\pi}$, so all we should be interested in finding is $\frac{x}{\pi} = \frac{<\alpha|\psi>}{\pi}$, which should be doable by scalar multiplication of the two.

In [1]:
import strawberryfields as sf
from strawberryfields.ops import *

import numpy as np

In [24]:
alpha = 1+0.5j
r = np.abs(alpha)
phi = np.angle(alpha)

prog = sf.Program(2) # two modes: the one we're interested in, and the 
with prog.context as q:
    Coherent() | q[0]
    Coherent(r,phi) | q[1] # this is the state we care about finding

eng = sf.Engine('fock', backend_options={"cutoff_dim": 15})
result = eng.run(prog, shots=1, modes=None, compile_options={})
state = result.state

# The above is one circuit, it yields a 15x15x15x15 matrix. 
# Instead, we could use two circuits and get 2 15x15 matrices.


In [31]:
rho2 = np.einsum("ijij->ij", state.dm())
rho2h

array([[0.2865048+0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j],
       [0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j],
       [0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j],
       [0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
        0.       +0.j,

In [44]:
# two different circuits: 
prog1 = sf.Program(1) # two modes: the one we're interested in, and the 
prog2 = sf.Program(1)
with prog1.context as q:
    Coherent() | q[0]
    
with prog2.context as q:
    Coherent(r,phi) | q[0] # this is the state we care about finding
    

eng = sf.Engine('fock', backend_options={"cutoff_dim": 15})
result1 = eng.run(prog1)
result2 = eng.run(prog2)

state1 = result1.state
state2 = result2.state

In [45]:
state1.dm().shape

(15, 15)

In [39]:
mult = state1.dm()*state2.dm()

In [42]:
mult.shape

(15, 15)

In [43]:
mult

array([[ 0.2865048+0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       -0.j,  0.       -0.j,  0.       -0.j, -0.       +0.j,
        -0.       +0.j, -0.       +0.j, -0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       +0.j,  0.       +0.j],
       [ 0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       -0.j,  0.       -0.j,  0.       -0.j,
        -0.       +0.j, -0.       +0.j, -0.       +0.j, -0.       +0.j,
         0.       +0.j,  0.       +0.j,  0.       +0.j],
       [ 0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       +0.j,  0.       -0.j,  0.       -0.j,
         0.       -0.j, -0.       +0.j, -0.       +0.j, -0.       +0.j,
        -0.       +0.j,  0.       +0.j,  0.       +0.j],
       [ 0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       -0.j,
         0.       -0.j,  0.       -0.